In [ ]:
#스마트파킹 점수 산출
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_absolute_error, r2_score
import math

# 1. 데이터 로드 및 기본 전처리 (시간별 주차현황)

file_path = "서울시설공단_공영주차장 시간별 주차현황_20240331.csv"
df = pd.read_csv(file_path, encoding="cp949")

# "시간대" 컬럼을 datetime 형식으로 변환 ("YYYY-MM-DD HH")
df["시간대"] = pd.to_datetime(df["시간대"], format="%Y-%m-%d %H", errors='coerce')
df = df.dropna(subset=["시간대"])  # 변환 실패 행 제거

# 시간과 요일 변수 생성
df["시간"] = df["시간대"].dt.hour
df["요일"] = df["시간대"].dt.dayofweek  # 0=월요일, 6=일요일

# 주차장명 좌우 공백 제거 및 소문자 변환 (이름 매칭 용이)
df["주차장명"] = df["주차장명"].str.strip().str.lower()

# 2. 혼잡도(%) 계산 및 추가 변수 생성

df["혼잡도(%)"] = ((df["입차대수"] - df["출차대수"]) / df["주차면수"]) * 100
df["혼잡도(%)"] = df["혼잡도(%)"].clip(lower=0, upper=100)

# 그룹 기준: "주차장명", "요일", "시간"에 대해 지난주, 지지난주, 지지지난주 혼잡도 계산
df["지난주_혼잡도"] = df.groupby(["주차장명", "요일", "시간"])["혼잡도(%)"].shift(7 * 24)
df["지지난주_혼잡도"] = df.groupby(["주차장명", "요일", "시간"])["혼잡도(%)"].shift(2 * 7 * 24)
df["지지지난주_혼잡도"] = df.groupby(["주차장명", "요일", "시간"])["혼잡도(%)"].shift(3 * 7 * 24)

# 결측치 처리: 그룹 내 forward fill 후, 남은 NaN은 전체 혼잡도 평균값으로 채움
df["지난주_혼잡도"] = df.groupby(["주차장명", "요일", "시간"])["지난주_혼잡도"].ffill()
df["지지난주_혼잡도"] = df["지지난주_혼잡도"].fillna(df["지난주_혼잡도"])
df["지지지난주_혼잡도"] = df["지지지난주_혼잡도"].fillna(df["지난주_혼잡도"])
for col in ["지난주_혼잡도", "지지난주_혼잡도", "지지지난주_혼잡도"]:
    df[col].fillna(df["혼잡도(%)"].mean(), inplace=True)

# 불필요한 컬럼 제거
df = df.drop(columns=["시간대"])

# 데이터 타입 최적화
for col in df.select_dtypes(include=["int64", "float64"]).columns:
    df[col] = df[col].astype("float32")

# 3. 특징 선택 및 전처리 (혼잡도 예측 모델)

features = [
    "주차면수", "입차대수", "출차대수",
    "지난주_혼잡도", "지지난주_혼잡도", "지지지난주_혼잡도",
    "시간", "요일"
]
target = "혼잡도(%)"

X = df[features].copy()
y = df[target].copy()

# 범주형 변수 변환 (요일: OrdinalEncoder 사용)
ordinal_encoder = OrdinalEncoder()
X.loc[:, "요일"] = ordinal_encoder.fit_transform(X[["요일"]])

# 수치형 변수 스케일링 (전체 열에 대해 한 번에 적용)
scaler = StandardScaler()
numerical_features = [
    "주차면수", "입차대수", "출차대수",
    "지난주_혼잡도", "지지난주_혼잡도", "지지지난주_혼잡도", "시간"
]
X[numerical_features] = scaler.fit_transform(X[numerical_features])

# 학습 시 사용한 컬럼 순서 저장 (예측 시 재정렬에 사용)
fit_columns = X.columns.tolist()

# 데이터 분할 및 모델 학습
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
best_model = XGBRegressor(
    n_estimators=300,
    max_depth=10,
    learning_rate=0.02,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
best_model.fit(X_train, y_train)

# 모델 성능 평가
y_test_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)
error_percentage = (mae / y_test.mean()) * 100

performance_metrics = {
    "평균 절대 오차 (MAE)": [mae],
    "결정 계수 (R²)": [r2],
    "평균 오차율 (%)": [error_percentage]
}
performance_df = pd.DataFrame(performance_metrics)
print("XGBoost 모델 성능 평가 결과:")
print(performance_df.to_string(index=False))

# 4. 추천 시스템: 최종 추천 점수 계산
# 가천대학교 좌표 (위도 37.450, 경도 127.129)

gachon_lat = 37.450
gachon_lon = 127.129

# haversine 공식에 의한 거리 계산 함수 (km 단위)
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반지름 (km)
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    d_phi = np.radians(lat2 - lat1)
    d_lambda = np.radians(lon2 - lon1)
    a = np.sin(d_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(d_lambda / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# 정적 정보 CSV 로드 (좌표만 사용)
static_file_path = "서울시_공영주차장_최종.csv"
df_static = pd.read_csv(static_file_path, encoding="cp949")

# CSV가 이미 올바른 컬럼명을 가지고 있다고 가정하므로 rename은 생략하고, 문자열 전처리만 진행
df_static["주차장명"] = df_static["주차장명"].astype(str).str.strip().str.lower()

# 필요한 컬럼 확인 (여기서는 "주차장명", "위도", "경도"만 사용)
required_columns = ["주차장명", "위도", "경도"]
missing_cols = [col for col in required_columns if col not in df_static.columns]
if missing_cols:
    print("정적 데이터 CSV 파일에 다음 컬럼들이 누락되었습니다:", missing_cols)
    print("CSV 파일의 컬럼명을 확인하세요. 사용 가능한 컬럼:", df_static.columns.tolist())

# df_static 정보를 이용해 추천 데이터프레임 생성 (주차장명 기준 병합)
parking_lot_names = df["주차장명"].unique()
recommend_df = pd.merge(pd.DataFrame({"주차장명": parking_lot_names}),
                        df_static[["주차장명", "위도", "경도"]],
                        on="주차장명", how="left")

# 주차요금은 dummy data로 생성 (1000 ~ 5000 원)
np.random.seed(42)
recommend_df["주차요금"] = np.random.uniform(1000, 5000, size=len(recommend_df))

# 리뷰는 1~5 사이의 랜덤값으로 생성
recommend_df["리뷰"] = np.random.uniform(1, 5, size=len(recommend_df))

# 각 주차장의 좌표와 가천대학교 좌표 간의 거리를 계산 (km)
recommend_df["거리"] = recommend_df.apply(
    lambda row: haversine_distance(gachon_lat, gachon_lon, row["위도"], row["경도"]),
    axis=1
)

# 예측 혼잡도 계산 함수 (주차장명이 없는 경우 전체 평균 혼잡도를 fallback 값으로 사용)
def predict_congestion_for_parking(parking_name, weekday, hour):
    data = df[df["주차장명"] == parking_name.strip()].copy()
    if data.empty:
        return df["혼잡도(%)"].mean()
    filtered = data[(data["요일"] == weekday) & (data["시간"] == hour)]
    if filtered.empty:
        avg_vals = data[[
            "주차면수", "입차대수", "출차대수",
            "지난주_혼잡도", "지지난주_혼잡도", "지지지난주_혼잡도"
        ]].mean()
    else:
        avg_vals = filtered[[
            "주차면수", "입차대수", "출차대수",
            "지난주_혼잡도", "지지난주_혼잡도", "지지지난주_혼잡도"
        ]].mean()
    input_df = pd.DataFrame({
        "주차면수": [avg_vals["주차면수"]],
        "입차대수": [avg_vals["입차대수"]],
        "출차대수": [avg_vals["출차대수"]],
        "지난주_혼잡도": [avg_vals["지난주_혼잡도"]],
        "지지난주_혼잡도": [avg_vals["지지난주_혼잡도"]],
        "지지지난주_혼잡도": [avg_vals["지지지난주_혼잡도"]],
        "시간": [hour],
        "요일": [weekday]
    })
    input_df = input_df.reindex(columns=fit_columns)
    input_df.loc[:, "요일"] = ordinal_encoder.transform(input_df[["요일"]])
    input_df[numerical_features] = scaler.transform(input_df[numerical_features])
    return best_model.predict(input_df)[0]

# 사용자가 원하는 시간 (수요일, 14시)
user_weekday = 3  # 수요일
user_hour = 14

# 각 주차장별 예측 혼잡도 계산
recommend_df["예측혼잡도"] = recommend_df["주차장명"].apply(lambda x: predict_congestion_for_parking(x, user_weekday, user_hour))
# 혼잡도 점수: 100 - 예측혼잡도
recommend_df["혼잡도점수"] = np.clip(100 - recommend_df["예측혼잡도"], 0, 100)

# 각 요소에 대해 0~100 점수 정규화
max_distance = recommend_df["거리"].max()
min_distance = recommend_df["거리"].min()
recommend_df["거리점수"] = (max_distance - recommend_df["거리"]) / (max_distance - min_distance) * 100

max_fee = recommend_df["주차요금"].max()
min_fee = recommend_df["주차요금"].min()
recommend_df["요금점수"] = (max_fee - recommend_df["주차요금"]) / (max_fee - min_fee) * 100

recommend_df["리뷰점수"] = (recommend_df["리뷰"] - 1) / 4 * 100

# 가중치 설정 (수정가능)
w_congestion = 0.4
w_distance   = 0.2
w_fee        = 0.15
w_review     = 0.15

# 최종 추천 점수 계산
recommend_df["최종추천점수"] = (
    w_congestion * recommend_df["혼잡도점수"] +
    w_distance   * recommend_df["거리점수"] +
    w_fee        * recommend_df["요금점수"] +
    w_review     * recommend_df["리뷰점수"]
)

# 최종 추천 점수 내림차순 정렬 후, 주차장명과 최종추천점수 출력
recommend_df = recommend_df.sort_values(by="최종추천점수", ascending=False)
print("\n주차장 추천 결과:")
print(recommend_df[["주차장명", "최종추천점수"]].to_string(index=False))

#수서역남 : 위도경도 없음
#수락산역, 화랑대역, 적선노외, 학여울역, 수락산역(폐쇄), 암사1동 : 최종csv에 없음